In [1]:
import os, math
import pandas as pd

In [2]:
blogPath = "./blogs/"
pathList = os.listdir(path=blogPath)
blogSampleSize = 50

In [3]:
pathList[0].split(".")

['1000331', 'female', '37', 'indUnk', 'Leo', 'xml']

In [4]:
def getOccupation(lst):
    rtrn = []
    for person in lst:
        rtrn.append(person.split(".")[3])
    return rtrn

occupations = getOccupation(pathList)[:blogSampleSize]
occupationDict = {}
occupationCounter = 0
for occ in occupations:
    if occ not in occupationDict:
        occupationDict[occ] = str(occupationCounter)
        occupationCounter += 1

In [5]:
blogs = []
for ind in range(blogSampleSize):
    file = open(blogPath+pathList[ind], "rb")
    posts = []
    for line in file:
        text = line.decode("utf-8", errors="ignore")
        text = text.strip()
        if text != "" and text[0] != "<":
            posts.append(text)
    blogs.append(" ".join(posts))
    file.close()


In [6]:
# splits up a string into a list of words.
# str -> [str]
def getWords(str):
    words = []
    
    str = str+"."
    lastInd = 0
    for ind in range(len(str)):
        if not (str[ind].isalpha()):
            if lastInd == ind:
                lastInd += 1
                continue
            else:
                words.append(str[lastInd:ind].lower())
                lastInd = ind+1
    
    return words
            

In [7]:
dicts = []
allWordsSeen = {}
for blog in blogs:
    newDict = {}
    dicts.append(newDict)
    wordList = getWords(blog)
    for word in wordList:
#         ADD TO THIS PERSON'S DICT
        if word not in newDict:
            newDict[word] = 1
        else:
            newDict[word] += 1
#         ADD TO ALL WORDS SEEN
        if word not in allWordsSeen:
            allWordsSeen[word] = 1
        else:
            allWordsSeen[word] += 1



In [8]:
totalPeople = len(dicts)
wordsToRemove = []
for word in allWordsSeen:
    count = allWordsSeen[word]
    
    if ((count / totalPeople) > .50) or ((count / totalPeople) < .01):
#         print(word)
#         del allWordsSeen[word]
        wordsToRemove.append(word)
        for person in dicts:
            if word in person:
                del person[word]
    else:
        for person in dicts:
            if word not in person:
                person[word] = 0

for word in wordsToRemove:
    del allWordsSeen[word]

In [9]:
dictOfDicts = {}
for ind in range(blogSampleSize):
    dictOfDicts[pathList[ind]] = dicts[ind]

In [10]:
tdm = pd.DataFrame(dictOfDicts, dtype=float)
# tdm = tdm.transpose()
# tdm

In [11]:
def termFrequency(person):
    return .5 + (.5*person/person.max())

In [12]:
# (This step scales really badly)

IDF_Dict = {}
for word in allWordsSeen:
    personCount = 0
    for person in tdm:
        if tdm[person][word] > 0:
            personCount += 1
    if personCount > 0:
        IDF_Dict[word] = math.log(blogSampleSize / personCount)

In [13]:
# IDF_Dict

In [14]:
for person in tdm:
    tdm[person] = termFrequency(tdm[person])
    for i in range(len(person)):
        tdm[person][i] = tdm[person][i] * IDF_Dict[tdm[person].keys()[i]]

In [15]:
# tdm

In [16]:
# cosDist = sum(ser1*ser2)/(sqrt(sum(ser1^2)) * sqrt(sum(ser2^2)))
def cosDist(ser1, ser2):
    numerator = ser1.dot(ser2)
    ser1Denominator = 0
    for val in ser1:
        ser1Denominator += math.pow(val, 2)
    ser1Denominator = math.sqrt(ser1Denominator)
    ser2Denominator = 0
    for val in ser2:
        ser2Denominator += math.pow(val, 2)
    ser2Denominator = math.sqrt(ser2Denominator)
    return 1 - numerator/(ser1Denominator*ser2Denominator)

In [17]:
keys = tdm.keys()

In [30]:
# (This step scales really badly)

similarityDict = {}
for person in tdm:
    similarityDict[person] = {}
    
for ind in range(len(keys)):
    for i in range(len(keys)):
        dist = cosDist(tdm[keys[ind]], tdm[keys[i]])
        similarityDict[keys[ind]][keys[i]] = ((dist < .0003) and (dist > .000000001))
#         if ((dist < .00007) and (dist > .000000001)):
#             print(dist)
#             print(i)
#             print(ind)

In [31]:
similarityFrame = pd.DataFrame(similarityDict)

In [32]:
# similarityFrame

In [33]:
# similarityFrame.to_json(path_or_buf="./similarity.json")

In [34]:
# similarityFrame = similarityFrame.transpose()
# similarityFrame.to_csv(path_or_buf="./similarity.csv", index_label="person", na_rep="undefined")

In [35]:
file = open("writeTest.json", "w")
file.write("{")
file.write('"nodes":[')
for i in range(len(tdm.keys())-1):
    file.write('{"id": "'+tdm.keys()[i]+'", "group":'+ occupationDict[occupations[i]] +'},')
file.write('{"id": "'+tdm.keys()[len(tdm.keys())-1]+'", "group": 1'+ occupationDict[occupations[i]] +'}')
file.write("],")

file.write('"links":[')
for person in similarityFrame:
    for relation in similarityFrame[person].keys():
        if similarityFrame[person][relation]:
            file.write('{"source": "'+person+'", "target": "'+relation+'", "value": 1}, \n')
file.write("]")
file.write("}")
file.close()